<a href="https://colab.research.google.com/github/Hugoloda/liukkvyufyu/blob/main/C%C3%B3digo_Bueno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En la primera celda de código importo el módulo os para interactuar con el sistema operativo ya que google collab fue actualizado y no exporta los códigos de la version que utilizamos.

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS']='1'

En la segunda celda importo la libreria de tensorflow para trabajar con los modelos entrenados con las fotos proporcionadas. También comprobamos las etiqueta que tiene cada una de las clasificaciones del modelo.

In [ ]:
from tensorflow import keras
# Carga un modelo Keras pre-entrenado desde la ruta especificada en Google Drive
# compile=False indica que no se compilará el modelo, asumiendo que ya está listo para inferencia
model=keras.models.load_model('/content/drive/MyDrive/converted_keras/keras_model.h5', compile=(False))
# Lee las etiquetas (nombres de clases) del modelo desde un archivo de texto
# Cada línea del archivo se convierte en un elemento de la lista 'nombres_clases'
nombres_clases=open('/content/drive/MyDrive/converted_keras/labels.txt','r').readlines()
print(nombres_clases)

['0 Gatitos\n', '1 Perros\n']


En esta última celda que contiene la mayoría del código se realizan diferentes procesos.

##Resumen de que contiene:

1º Importo las últimas librerias necesaria para el trabajo y modificación de las imágenes y también para las operaciones numéricas.

2º Defino la función predecir_imagen dada su ruta, y después una vez cargada la imagen la convierto en formato RGB (rojo, verde, azul) y defino el tamaño de la imagen a 224x224 pixeles desde el centro de la imagen para que asi el módelo pueda reconocerla.

3º Después convierto la imagen a un array que viene de numpy y convierto los valores de esta a valores entre -1 y 1 que son los que espera el modelo entrenado.

4º Al resultado coge el valor de la imagen una vez predecida por el modelo entrenado que ya ha analizado el lote con la imagen ajustada para que pueda leerla y consigo el índice de la clase con la cual el sistema diceque hay mayor probabilidad de ser, obtengo el nombre de dicha clase y obtengo el valor de la probabilidad de predicción. Y me devuelvo el nombre de la etiqueta que ya hemos identificado junto con su probabilidad.

5º Tras esto analizamos toda la lista de fotos que hemos adaptado y vemos si son gatos o perros con sus etiquetas y se llama a la función predecir_imagen para obtener la predicción y la probabilidad.

6º Se imprime todas las variables y datos como la catidad de aciertos o la probabilidad de que cada foto sea de la clase que el modelo cree que es.

In [ ]:
from PIL import Image, ImageOps
import numpy as np

def predecir_imagen(ruta_imagen):
    image = Image.open(ruta_imagen).convert("RGB")
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Prepara el array de la imagen para la predicción del modelo
    # Crea un lote de imágenes (en este caso, un lote de una sola imagen) con el formato esperado por el modelo (1, alto, ancho, canales)
    lote_imagenes = np.ndarray(shape=(1, 224, 224, 3,), dtype=np.float32)
    # Asigna la imagen normalizada al lote
    lote_imagenes[0] = normalized_image_array

    # Realiza la predicción usando el modelo cargado
    resultado = model.predict(lote_imagenes)

    # Obtiene el índice de la clase con la mayor probabilidad (la predicción del modelo)
    indice = np.argmax(resultado[0])
    # Obtiene la etiqueta de la clase predicha de la lista de nombres de clases
    etiqueta = nombres_clases[indice]
    # Obtiene la probabilidad asociada a la clase predicha
    probabilidad = resultado[0][indice]
    # Retorna la etiqueta y la probabilidad de la predicción
    return etiqueta, probabilidad

lista_archivos = os.listdir('/content/drive/MyDrive/gatos vs perros/cats_dogs_light/test')
total_predicciones = 0
aciertos = 0
media_probabilidad_aciertos = 0.0
predicciones_incorrectas = [] # Lista para almacenar información de predicciones incorrectas

print("\n--- Realizando predicciones ---")

for nombre_archivo in lista_archivos:
    # Determina la etiqueta esperada para el archivo basándose en su nombre
    if 'cat' in nombre_archivo:
        etiqueta_esperada = "0 Gatitos\n"
    elif 'dog' in nombre_archivo:
        etiqueta_esperada = "1 Perros\n"
    # Si el nombre no contiene ni 'cat' ni 'dog', salta al siguiente archivo
    else:
        continue

    total_predicciones += 1

    ruta_imagen = os.path.join('/content/drive/MyDrive/gatos vs perros/cats_dogs_light/test', nombre_archivo)

    # Llama a la función predecir_imagen para obtener la predicción y la probabilidad
    etiqueta_predicha, probabilidad = predecir_imagen(ruta_imagen)

    print(f"Archivo: {nombre_archivo}, Predicción: {etiqueta_predicha.strip()}, Probabilidad: {probabilidad:.4f}, Esperado: {etiqueta_esperada.strip()}")

    if etiqueta_predicha == etiqueta_esperada:
        aciertos += 1
        media_probabilidad_aciertos += probabilidad
    else:
        # Si es incorrecta, guarda la información del error en la lista
        info_error= {"archivo": nombre_archivo, "prediccion": etiqueta_predicha.strip(),
             "probabilidad": probabilidad}
        predicciones_incorrectas.append(info_error)

print("\n--- Resumen de predicciones ---")
print(f"Total de imágenes procesadas: {total_predicciones}")
print(f"Aciertos: {aciertos}")

if total_predicciones > 0:
    precision = (aciertos / total_predicciones) * 100
    print(f"Precisión: {precision:.2f}%")
else:
    print("No hay imágenes para procesar.")

if aciertos > 0:
    media_probabilidad_aciertos /= aciertos
    print(f"Probabilidad media de aciertos: {media_probabilidad_aciertos:.4f}")
else:
    print("No hubo aciertos.")

if predicciones_incorrectas:
    print("\n--- Predicciones incorrectas ---")
    for error in predicciones_incorrectas:
        print(f"Archivo: {error['archivo']}, Predicción: {error['prediccion']}, Probabilidad: {error['probabilidad']:.4f}")
else:
    print("\nTodas las predicciones fueron correctas")


--- Realizando predicciones ---
1/1 [==============================] - 1s 1s/step
Archivo: dog.9791.jpg, Predicción: 1 Perros, Probabilidad: 1.0000, Esperado: 1 Perros
1/1 [==============================] - 0s 42ms/step
Archivo: dog.9804.jpg, Predicción: 1 Perros, Probabilidad: 0.9460, Esperado: 1 Perros
1/1 [==============================] - 0s 42ms/step
Archivo: dog.9806.jpg, Predicción: 1 Perros, Probabilidad: 1.0000, Esperado: 1 Perros
1/1 [==============================] - 0s 47ms/step
Archivo: dog.9780.jpg, Predicción: 1 Perros, Probabilidad: 1.0000, Esperado: 1 Perros
1/1 [==============================] - 0s 99ms/step
Archivo: dog.9803.jpg, Predicción: 1 Perros, Probabilidad: 0.9562, Esperado: 1 Perros
1/1 [==============================] - 0s 56ms/step
Archivo: dog.9795.jpg, Predicción: 1 Perros, Probabilidad: 1.0000, Esperado: 1 Perros
1/1 [==============================] - 0s 71ms/step
Archivo: dog.9778.jpg, Predicción: 1 Perros, Probabilidad: 1.0000, Esperado: 1 Perros
1/1